# dependencies and hyperparameters

In [1]:
#import dependencies
import torch
import torch.nn as nn
from PIL import Image
import pandas as pd
from torch.utils.data import DataLoader
from torch.optim import Adam
from torchvision.transforms import ToTensor
from torchvision import datasets

In [2]:
#hyperparameters
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
epochs = 10

# dataset

I decided to not use the built in mnist dataset in torch since I want to learn how to upload files into google colab and such. It's not that complicated but nice practice. Also did not include any file upload features as uploading mnist-large files takes upwards of 30min

In [ ]:
#temporary while I try finding a way to transform the csv file to the correct dimensionality
train = datasets.MNIST(root="data", download=True, train=True, transform=ToTensor())
trainset = DataLoader(train, 32)

#1, 28, 28 - classes: 0,1,2,3,4,5,6,7,8,9 

In [ ]:
#DONT USE
#to be used when i have implemented a transformation of the csv file to fit the format needed
#1 FOR USE ON COLAB WEB
#upload files to your drive and use the correct paths

from google.colab import drive
drive.mount('/content/drive')

#Set path to the training and validation data

path_train = "/content/drive/MyDrive/mnist_neural_network/mnist_train.csv"

In [47]:
# DONT USE
#to be used when i have implemented a transformation of the csv file to fit the format needed
#2 FOR USE ON LOCAL MACHINE
path_train = "/dataset/mnist_test.csv"

In [6]:
#DONT USE
#to be used when i have implemented a transformation of the csv file to fit the format needed
#read the files
train = pd.read_csv(path_train)

#shorten the dataset
train = train.iloc[:15001]

In [ ]:
#DONT USE
#to be used when i have implemented a transformation of the csv file to fit the format needed
#convert the dataframe representations of the data to pytorch tensors and set correct device
train = torch.from_numpy(train.values).float().to(device)
trainset = DataLoader(train, 32)

# model

In [4]:
#image classifier network
class ImageClassifier(nn.Module):
    def __init__(self):
      super().__init__()
      self.model = nn.Sequential(
          nn.Conv2d(1, 32, (3, 3)), # 1 input channel (greyscale), 32 output channel
          nn.ReLU(),
          nn.Conv2d(32, 64, (3, 3)), # 32 input channels from previous layer
          nn.ReLU(),
          nn.Conv2d(64, 64, (3, 3)), # 64 input channels from previous layer
          nn.ReLU(),
          nn.Flatten(),
          nn.Linear(64*(28-6)*(28-6), 10)
      ) 

    def forward(self, x):
      return self.model(x)

In [5]:
# instance of neural network, loss and optimizer
clf = ImageClassifier().to(device)
opt = Adam(clf.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
#training
if __name__ == "__main__":
  for epoch in range(epochs):
    for batch in trainset:
      x,y = batch
      x,y = x.to(device), y.to(device)
      logits = clf(x)
      loss = loss_fn(logits, y)

      opt.zero_grad(set_to_none=True)
      loss.backward()
      opt.step()

    print(f"for epoch: {epoch}, the loss is {loss.item()}")

In [13]:
#save the model
with open('model_state.pt', 'wb') as f:
  torch.save(clf.state_dict(), f)

# test the model

In [ ]:
#open a saved model
with open('model_state.pt', 'rb') as f: 
    clf.load_state_dict(torch.load(f))  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path_image = '/content/drive/MyDrive/mnist_neural_network/2.jpeg'
img = Image.open(path_image) 
img_tensor = ToTensor()(img).unsqueeze(0).to(device)

print(torch.argmax(clf(img_tensor)))